In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('./xyz_sales.db')

cursor = conn.cursor()

In [4]:
#get each table, assign it into a dataframe

sales_table_query = "SELECT * FROM sales"
orders_table_query = "SELECT * FROM orders"
items_table_query = "SELECT * FROM items"
customers_table_query = "SELECT * FROM customers"


sales_table = pd.read_sql_query(sales_table_query, conn)
orders_table_raw = pd.read_sql_query(orders_table_query, conn)
items_table = pd.read_sql_query(items_table_query, conn)
customers_table = pd.read_sql_query(customers_table_query, conn)

In [5]:
sales_table

,sales_id,customer_id
0,1,10
1,2,85
2,3,42
3,4,93
4,5,75
...,...,...
495,496,22
496,497,68
497,498,77
498,499,10


In [6]:
orders_table_raw

,order_id,sales_id,item_id,quantity
0,1,1,1,2.0
1,2,1,2,NaN
2,3,1,3,1.0
3,4,2,1,1.0
4,5,2,2,1.0
...,...,...,...,...
1495,1496,499,2,NaN
1496,1497,499,3,2.0
1497,1498,500,1,1.0
1498,1499,500,2,2.0


In [7]:
orders_table = orders_table_raw.dropna()
orders_table["quantity"] = orders_table["quantity"].astype(int)
orders_table

C:\Users\CARLO\AppData\Local\Temp\ipykernel_8276\709546425.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_table["quantity"] = orders_table["quantity"].astype(int)


,order_id,sales_id,item_id,quantity
0,1,1,1,2
2,3,1,3,1
3,4,2,1,1
4,5,2,2,1
5,6,2,3,2
...,...,...,...,...
1494,1495,499,1,2
1496,1497,499,3,2
1497,1498,500,1,1
1498,1499,500,2,2


In [9]:
items_table

,item_id,item_name
0,1,x
1,2,y
2,3,z


In [10]:
customers_table

,customer_id,age
0,1,38
1,2,43
2,3,13
3,4,39
4,5,18
...,...,...
95,96,70
96,97,23
97,98,41
98,99,56


In [11]:
customers_mask = (customers_table['age']>= 18) & (customers_table['age'] <= 35)
customers_filtered = customers_table[customers_mask]
customers_filtered

,customer_id,age
4,5,18
6,7,30
8,9,25
15,16,19
21,22,18
27,28,21
28,29,30
30,31,22
33,34,22
34,35,20


In [12]:
orders_with_sales_table = pd.merge(orders_table, sales_table, on="sales_id", how="inner")
orders_with_sales_table

,order_id,sales_id,item_id,quantity,customer_id
0,1,1,1,2,10
1,3,1,3,1,10
2,4,2,1,1,85
3,5,2,2,1,85
4,6,2,3,2,85
...,...,...,...,...,...
942,1495,499,1,2,10
943,1497,499,3,2,10
944,1498,500,1,1,31
945,1499,500,2,2,31


In [13]:
customers_with_orders_and_sales_table = pd.merge(customers_filtered, orders_with_sales_table, on="customer_id", how="inner")
print(customers_with_orders_and_sales_table)

,customer_id,age,order_id,sales_id,item_id,quantity
0,5,18,511,171,1,1
1,5,18,512,171,2,3
2,5,18,513,171,3,1
3,5,18,547,183,1,1
4,5,18,549,183,3,2
...,...,...,...,...,...,...
233,100,30,760,254,1,1
234,100,30,761,254,2,1
235,100,30,762,254,3,2
236,100,30,966,322,3,1


In [35]:
combined_table_with_label = pd.merge(customers_with_orders_and_sales_table, items_table, on="item_id", how="inner")
combined_table_with_label

,customer_id,age,order_id,sales_id,item_id,quantity,item_name
0,5,18,511,171,1,1,x
1,5,18,547,183,1,1,x
2,5,18,1213,405,1,1,x
3,7,30,106,36,1,2,x
4,7,30,157,53,1,1,x
...,...,...,...,...,...,...,...
233,97,23,441,147,3,2,z
234,97,23,1389,463,3,1,z
235,100,30,762,254,3,2,z
236,100,30,966,322,3,1,z


In [37]:
final_df = combined_table_with_label.groupby(['customer_id','age','item_name'])['quantity'].sum().reset_index()
final_df

,customer_id,age,item_name,quantity
0,5,18,x,3
1,5,18,y,3
2,5,18,z,4
3,7,30,x,14
4,7,30,y,19
...,...,...,...,...
65,97,23,y,2
66,97,23,z,3
67,100,30,x,1
68,100,30,y,1


In [18]:
output_headers = ['Customer','Age','Item','Quantity']
final_df.to_csv('output_pandas.csv',index=False, header=output_headers)

In [ ]:
# Now let's try to do the same thing above using a sql query

In [31]:
sql_solution_string = "SELECT customers.customer_id, \
            		customers.age, \
            		items.item_name, \
            		sum(orders.quantity) \
                    FROM orders \
                    INNER JOIN sales ON orders.sales_id = sales.sales_id \
                    INNER JOIN customers ON sales.customer_id = customers.customer_id \
                    INNER JOIN items ON orders.item_id = items.item_id \
                    WHERE customers.age >= 18 and customers.age <=35 \
                    GROUP BY customers.customer_id, customers.age, items.item_name \
                    HAVING sum(orders.quantity) > 0"

In [32]:
sql_solution_table = pd.read_sql_query(sql_solution_string, conn)

In [33]:
sql_solution_table

,customer_id,age,item_name,sum(orders.quantity)
0,5,18,x,3
1,5,18,y,3
2,5,18,z,4
3,7,30,x,14
4,7,30,y,19
...,...,...,...,...
65,97,23,y,2
66,97,23,z,3
67,100,30,x,1
68,100,30,y,1


In [38]:
sql_solution_table.to_csv('output_sql.csv',index=False, header=output_headers)

In [39]:
sql_df

,0,1,2,3
0,5,18,x,3
1,5,18,y,3
2,5,18,z,4
3,7,30,x,14
4,7,30,y,19
...,...,...,...,...
65,97,23,y,2
66,97,23,z,3
67,100,30,x,1
68,100,30,y,1
